In [1]:
import os
import math
import random
import argparse
import ast
import copy
import warnings
from pathlib import Path
from typing import Tuple, Sequence, Union
from collections import Counter, defaultdict
from itertools import chain, combinations

import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer

from utils.Spambase.split_data import split_data_equal
from utils.aggregate_functions import aggregate_lr_models, FederatedForest
from utils.evaluate_coalitions_new import evaluate_coalitions2
from utils.DecisionTree import DecisionTree
from utils.nash1 import find_nash_equilibria_v2

In [2]:
def train_models_fedlr(partitions, random_seed, X_test, y_test, max_iter):
    client_models = []
    client_global_accuracies = []
    
    for X_i, y_i in partitions:
        nan_mask = ~np.isnan(X_i).any(axis=1)
        X_clean = X_i[nan_mask]
        y_clean = y_i[nan_mask]
        if len(y_clean) == 0:
            client_models.append(None)
            client_global_accuracies.append(None)
            continue
        
        model = LogisticRegression(random_state=random_seed, max_iter=max_iter)
        try:
            local_scaler = StandardScaler()
            model.fit(local_scaler.fit_transform(X_clean), y_clean)
            client_models.append(model)
            client_global_accuracies.append(model.score(X_test, y_test))
        except Exception as e:
            client_models.append(None)
            client_global_accuracies.append(None)
    
    return client_models, client_global_accuracies

In [3]:
def train_models_fedfor(partitions, X_test, y_test, max_depth, trial_seed):
    client_models = []
    client_global_accuracies = {}
    
    for i, (X_i, y_i) in enumerate(partitions):
        model = DecisionTreeClassifier(
            max_depth=max_depth,
            random_state=trial_seed
        )
        model.fit(X_i, y_i)
        client_models.append(model)

        # Evaluate directly on the fixed global test set
        y_pred = model.predict(X_test)
        client_global_accuracies[i] = np.mean(y_pred == y_test)
    
    return client_models, client_global_accuracies

### FedLR _ Spambase

In [4]:
file_path = '/Users/abbaszal/Documents/Thesis_Project_Spambase/data/spambase.data'  # Adjust the path as needed
df = pd.read_csv(file_path, header=None)

In [ ]:
import joblib
BUNDLE_PATH = '/.../tree_shape_no_level_none_delta_noN_leaveout_spambase.joblib'

In [8]:
X_full = df.iloc[:, :-1].to_numpy()
y_full = df.iloc[:, -1].to_numpy()

In [ ]:
eps = 1e-8
n_trials = 100
n_clients_list = [10,20,30,40,50,60,70,80,90,100]
base_seed = 42
max_iters = [100]
approach = 'fedlr'

save_dir = "/.../spambase_fedlr_tree"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
_bundle = joblib.load(BUNDLE_PATH)
_PIPE = _bundle["pipeline"]
_SAVED_FEATURES = _bundle["features"]


_META = {
    "selected_features": list(_SAVED_FEATURES),
    "featureset": "shape_no_level",
    "include_n_clients": False,  
    "target_mode": "delta",
    "logit_target": False,
}

def _sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def _base_stats_from_accs(accs_1d):
    v = np.asarray(accs_1d, dtype=float)
    return {
        'mean': float(np.mean(v)),
        'median': float(np.median(v)),
        'max': float(np.max(v)),
        'percentile_90': float(np.percentile(v, 90)),
        'percentile_75': float(np.percentile(v, 75)),
        'percentile_25': float(np.percentile(v, 25)),
        'percentile_10': float(np.percentile(v, 10)),
        'min': float(np.min(v)),
    }

def _features_from_client_accs(accs_dict, meta, n_clients=None):

    vals = np.array([accs_dict[k] for k in sorted(accs_dict.keys())], dtype=float)
    base = _base_stats_from_accs(vals)

    # Absolute stats (always compute them first)
    row = {
        'mean': base['mean'], 'median': base['median'], 'max': base['max'],
        'percentile_90': base['percentile_90'], 'percentile_75': base['percentile_75'],
        'percentile_25': base['percentile_25'], 'percentile_10': base['percentile_10'], 'min': base['min']
    }
    df_feat = pd.DataFrame([row])
    if meta.get('featureset') in ('shape', 'shape_no_level'):
        df_feat['std'] = df_feat[['min','percentile_10','percentile_25','median','percentile_75','percentile_90','max']].std(axis=1)
        for c in ['median','max','percentile_90','percentile_75','percentile_25','percentile_10','min']:
            df_feat[f'{c}_dm'] = df_feat[c] - df_feat['mean']

    if meta.get('include_n_clients', False):
        df_feat['n_clients'] = n_clients if n_clients is not None else np.nan

    feat_cols = meta['selected_features']
    missing = [c for c in feat_cols if c not in df_feat.columns]
    if missing:
        raise ValueError(f"Feature mismatch. Missing columns for model: {missing}")
    return df_feat[feat_cols], float(base['mean'])

def predict_global_from_loaded(accs_dict, n_clients=None):
    X, mean_val = _features_from_client_accs(accs_dict, _META, n_clients)
    y_model = _PIPE.predict(X)

    if _META.get('target_mode', 'prob') == 'delta':
        y = float(y_model[0] + mean_val)  
    else:
        y = float(_sigmoid(y_model[0]) if _META.get('logit_target', False) else y_model[0])


    return float(np.clip(y, 0.0, 1.0))


def seed_for(n_clients: int, max_iter: int, trial: int) -> int:
    rnd = random.Random(base_seed + 1000*max_iter + 37*n_clients + 17*trial + 7919)
    rc = rnd.randint(0, 10_000_000)
    return base_seed + (trial - 1) + 1000*max_iter + 37*n_clients + 2*rc



all_results = []
best_vs_predicted = []

for n_clients in n_clients_list:
    print(f"\n> n_clients = {n_clients}")

    for max_iter in max_iters:
        print(f"  max_iter = {max_iter}")
        complete_static = Counter()
        lottery_count = 0

        for trial in range(1, n_trials + 1):
            trial_seed = seed_for(n_clients, max_iter, trial)
            random.seed(trial_seed)
            np.random.seed(trial_seed)

            X_train_pool, X_test, y_train_pool, y_test = train_test_split(
                X_full, y_full, test_size=0.2, random_state=trial_seed, stratify=y_full
            )

            scaler = StandardScaler()
            X_train_pool_scaled = scaler.fit_transform(X_train_pool)
            X_test_scaled = scaler.transform(X_test)

            partitions = split_data_equal(
                X_train_pool_scaled, y_train_pool,
                n_clients=n_clients,
                shuffle=True,
                random_seed=trial_seed
            )

            client_models, client_accs = train_models_fedlr(
                partitions=partitions,
                random_seed=trial_seed,
                X_test=X_test_scaled,
                y_test=y_test,
                max_iter=max_iter
            )

            if isinstance(client_accs, dict):
                client_accs_dict = {int(k): float(v) for k, v in client_accs.items()}
                client_acc_list = [client_accs_dict[i] for i in sorted(client_accs_dict)]
            else:
                client_acc_list = [float(a) for a in client_accs]
                client_accs_dict = {i: a for i, a in enumerate(client_acc_list)}


            df_res = evaluate_coalitions2(
                client_models=client_models,
                client_global_accuracies=client_accs_dict,
                n_clients=n_clients,
                aggregator_func=aggregate_lr_models,
                X_test=X_test_scaled,
                y_test=y_test,
                corrupt_client_indices=[],
                approach=approach
            )


            df_ne = find_nash_equilibria_v2(df_res)
            if not df_ne.empty:
                for coalition in df_ne.index:
                    complete_static[coalition] += 1

            accs_simple = {int(cid): float(acc) for cid, acc in client_accs_dict.items()}
            payoff_f = predict_global_from_loaded(accs_simple, n_clients=n_clients)

            # Actual global accuracy for the grand coalition
            full_coalition_str = "1" * n_clients
            row_full = df_res.loc[df_res['Combination'] == full_coalition_str, 'Global Accuracy']
            actual_global_acc = row_full.iloc[0] if not row_full.empty else np.nan

            # Collect results
            all_results.append({
                'n_clients':        n_clients,
                'max_iter':         max_iter,
                'trial':            trial,
                'predicted_global': payoff_f,
                'actual_global':    actual_global_acc
            })

            # Best Client Accuracy vs Prediction
            best_client_acc = float(np.nanmax(client_acc_list)) if len(client_acc_list) else np.nan
            best_vs_predicted.append({
                'n_clients': n_clients,
                'trial': trial,
                'best_client_acc': best_client_acc,
                'predicted_global': payoff_f,
                'actual_global': actual_global_acc
            })

            # Lottery check
            has_incentive = any(acc > payoff_f for acc in client_acc_list)
            if not has_incentive:
                lottery_count += 1


        complete_count = sum(complete_static.values())
        counts_df = pd.DataFrame([{
            'n_clients': n_clients,
            'max_iter': max_iter,
            'Complete_Occurrences': complete_count,
            'Lottery_Occurrences': lottery_count
        }])
        fname = f"Nash_Counts_{approach}_nclients_{n_clients}_maxiter_{max_iter}.csv"
        out_path = os.path.join(save_dir, fname)
        counts_df.to_csv(out_path, index=False)



results_df = pd.DataFrame(all_results)
results_path = os.path.join(save_dir, "predicted_vs_actual_all_trials.csv")
results_df.to_csv(results_path, index=False)



df_best_pred = pd.DataFrame(best_vs_predicted)
best_pred_path = os.path.join(save_dir, "best_client_vs_predicted_global.csv")
df_best_pred.to_csv(best_pred_path, index=False)

In [ ]:
results_df = pd.DataFrame(all_results)
results_df = results_df.dropna(subset=['actual_global'])
results_df['error'] = results_df['predicted_global'] - results_df['actual_global']
optimistic_count = (results_df['error'] > 0).sum()
pessimistic_count = (results_df['error'] < 0).sum()
avg_optimism = results_df.loc[results_df['error'] > 0, 'error'].mean()
avg_pessimism = results_df.loc[results_df['error'] < 0, 'error'].mean()

# RMSE
rmse = np.sqrt((results_df['error'] ** 2).mean())

print("\n=== Estimator Bias Summary ===")
print(f"Total trials: {len(results_df)}")
print(f"Optimistic trials: {optimistic_count} ({optimistic_count/len(results_df):.2%})")
print(f"Pessimistic trials: {pessimistic_count} ({pessimistic_count/len(results_df):.2%})")
print(f"Average optimism bias: {avg_optimism:.4f}")
print(f"Average pessimism bias: {avg_pessimism:.4f}")
print(f"RMSE: {rmse:.4f}")
error_report_path = os.path.join(save_dir, "estimator_error_report.csv")
results_df.to_csv(error_report_path, index=False)


In [ ]:
import os, glob
import pandas as pd
import matplotlib.pyplot as plt

pattern = os.path.join(save_dir, f"Nash_Counts_{approach}_nclients_*_maxiter_{max_iters[0]}.csv")
files = sorted(glob.glob(pattern))


counts_all = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
counts_all = counts_all.sort_values("n_clients")


plt.figure(figsize=(10, 6))
plt.plot(counts_all["n_clients"], counts_all["Complete_Occurrences"], marker="o", label="Complete information")
plt.plot(counts_all["n_clients"], counts_all["Lottery_Occurrences"], marker="s", label="Lottery occurrences")
plt.xlabel("Number of clients")
plt.ylabel(f"Occurrences (out of {n_trials})")
plt.title(f"Spmabase FedLR: Complete information vs Lottery rates across clients")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "occurrences_vs_clients_counts.png"), dpi=200)
plt.show()


counts_all["Complete_Rate"]  = counts_all["Complete_Occurrences"]  / n_trials
counts_all["Lottery_Rate"] = counts_all["Lottery_Occurrences"] / n_trials

plt.figure()
plt.plot(counts_all["n_clients"], counts_all["Complete_Rate"], marker="o", label="Complete rate")
plt.plot(counts_all["n_clients"], counts_all["Lottery_Rate"], marker="s", label="Lottery rate")
plt.xlabel("Number of clients")
plt.ylabel("Rate")
plt.ylim(0, 1)
plt.title(f"Spmabase: Complete information vs Lottery rates across clients")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

counts_all


### FedFor _ Spambase

In [14]:
file_path = '/Users/abbaszal/Documents/Thesis_Project_Spambase/data/spambase.data'  # adjust if needed
df = pd.read_csv(file_path, header=None)

In [ ]:
BUNDLE_PATH = ".../linear_fedfor_delta_noN/linear_shape_no_level_none_delta_noN_leaveout_spambase.joblib"

In [ ]:
eps = 1e-8
n_clients_list = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
n_trials = 100
base_seed = 42
max_depths = [100]     
approach = 'fedfor'

save_dir = "/.../spambase_fedfor_linear"
os.makedirs(save_dir, exist_ok=True)

In [17]:
X_full = df.iloc[:, :-1].to_numpy()
y_full = df.iloc[:, -1].to_numpy()

In [ ]:
_bundle = joblib.load(BUNDLE_PATH)
pipeline = _bundle["pipeline"]
_saved_features = list(_bundle["features"])


_META = {
    "selected_features": _saved_features,
    "featureset": "shape_no_level",
    "include_n_clients": False,   
    "target_mode": "delta",        
    "logit_target": False,          
}



def _base_stats_from_accs(accs_1d: np.ndarray) -> dict:
    
    v = np.asarray(accs_1d, dtype=float)
    return {
        'mean': float(np.mean(v)),
        'median': float(np.median(v)),
        'max': float(np.max(v)),
        'percentile_90': float(np.percentile(v, 90)),
        'percentile_75': float(np.percentile(v, 75)),
        'percentile_25': float(np.percentile(v, 25)),
        'percentile_10': float(np.percentile(v, 10)),
        'min': float(np.min(v)),
    }



def _features_from_client_accs(accs_dict: dict, meta: dict, n_clients: int | None = None) -> tuple[pd.DataFrame, float]:

    vals = np.array([accs_dict[k] for k in sorted(accs_dict.keys())], dtype=float)
    base = _base_stats_from_accs(vals)


    row = {
        'mean': base['mean'], 'median': base['median'], 'max': base['max'],
        'percentile_90': base['percentile_90'], 'percentile_75': base['percentile_75'],
        'percentile_25': base['percentile_25'], 'percentile_10': base['percentile_10'], 'min': base['min']
    }
    df_feat = pd.DataFrame([row])


    if meta.get('featureset') in ('shape', 'shape_no_level'):
        df_feat['std'] = df_feat[['min','percentile_10','percentile_25','median','percentile_75','percentile_90','max']].std(axis=1)
        for c in ['median','max','percentile_90','percentile_75','percentile_25','percentile_10','min']:
            df_feat[f'{c}_dm'] = df_feat[c] - df_feat['mean']

    
    if meta.get('include_n_clients', False):
        df_feat['n_clients'] = n_clients if n_clients is not None else np.nan

    feat_cols = meta['selected_features']

    return df_feat[feat_cols], float(base['mean'])

def predict_global_from_loaded(accs_dict: dict, n_clients: int | None = None) -> float:

    X, mean_val = _features_from_client_accs(accs_dict, _META, n_clients)
    y_model = float(pipeline.predict(X)[0])  
    y = y_model + mean_val                  
    return float(np.clip(y, 0.0, 1.0))




all_results = []
best_vs_predicted = []



for n_clients in n_clients_list:
    print(f"\n> n_clients = {n_clients}")

    for max_depth in max_depths:
        print(f"  max_depth = {max_depth}")
        counts_complete = Counter()
        lottery_count = 0

        for trial in range(1, n_trials + 1):
            trial_seed = base_seed + trial + 1000 * max_depth + 37 * n_clients
            random.seed(trial_seed)
            np.random.seed(trial_seed)


            X_train, X_test, y_train, y_test = train_test_split(
                X_full, y_full, test_size=0.2, random_state=trial_seed  
            )

            partitions = split_data_equal(
                X_train, y_train,
                n_clients=n_clients,
                shuffle=True,
                random_seed=trial_seed
            )

            client_models, client_accs = train_models_fedfor(
                partitions=partitions,
                X_test=X_test,
                y_test=y_test,
                max_depth=max_depth,
                trial_seed=trial_seed
            )

            df_res = evaluate_coalitions2(
                client_models=client_models,
                client_global_accuracies=client_accs,
                n_clients=n_clients,
                aggregator_func=FederatedForest,
                X_test=X_test,
                y_test=y_test,
                corrupt_client_indices=[],
                approach=approach
            )


            df_ne = find_nash_equilibria_v2(df_res)
            if not df_ne.empty:
                for coalition in df_ne.index:
                    counts_complete[coalition] += 1

            accs_simple = {int(cid): float(acc) for cid, acc in client_accs.items()}
            payoff_f = predict_global_from_loaded(accs_simple, n_clients=n_clients)

            # Actual Global Accuracy for grnd coalition
            full_coalition_str = "1" * n_clients
            row_full = df_res.loc[df_res['Combination'] == full_coalition_str, 'Global Accuracy']
            actual_global_acc = float(row_full.iloc[0]) if not row_full.empty else np.nan

    
            all_results.append({
                'n_clients': n_clients,
                'max_depth': max_depth,
                'trial': trial,
                'predicted_global': payoff_f,
                'actual_global': actual_global_acc
            })

            # Best client vs prediction
            best_client_acc = float(np.max(list(accs_simple.values()))) if accs_simple else np.nan
            best_vs_predicted.append({
                'n_clients': n_clients,
                'trial': trial,
                'best_client_acc': best_client_acc,
                'predicted_global': payoff_f,
                'actual_global': actual_global_acc
            })

            # Incentive check
            has_incentive = any(acc > payoff_f for acc in accs_simple.values())
            if not has_incentive:
                lottery_count += 1


        complete_count = sum(counts_complete.values())
        counts_df = pd.DataFrame([{
            'n_clients': n_clients,
            'max_iter': max_depth,
            'Complete_Occurrences': complete_count,
            'Lottery_Occurrences': lottery_count
        }])
        fname = f"Nash_Counts_{approach}_nclients_{n_clients}_maxiter_{max_depth}.csv"
        out_path = os.path.join(save_dir, fname)
        counts_df.to_csv(out_path, index=False)



results_df = pd.DataFrame(all_results)
results_path = os.path.join(save_dir, "predicted_vs_actual_all_trials.csv")
results_df.to_csv(results_path, index=False)


df_best_pred = pd.DataFrame(best_vs_predicted)
best_pred_path = os.path.join(save_dir, "best_client_vs_predicted_global.csv")
df_best_pred.to_csv(best_pred_path, index=False)



In [ ]:
results_df = pd.DataFrame(all_results)
results_df = results_df.dropna(subset=['actual_global'])
results_df['error'] = results_df['predicted_global'] - results_df['actual_global']
optimistic_count = (results_df['error'] > 0).sum()
pessimistic_count = (results_df['error'] < 0).sum()
avg_optimism = results_df.loc[results_df['error'] > 0, 'error'].mean()
avg_pessimism = results_df.loc[results_df['error'] < 0, 'error'].mean()

# RMSE
rmse = np.sqrt((results_df['error'] ** 2).mean())

print("\n=== Estimator Bias Summary ===")
print(f"Total trials: {len(results_df)}")
print(f"Optimistic trials: {optimistic_count} ({optimistic_count/len(results_df):.2%})")
print(f"Pessimistic trials: {pessimistic_count} ({pessimistic_count/len(results_df):.2%})")
print(f"Average optimism bias: {avg_optimism:.4f}")
print(f"Average pessimism bias: {avg_pessimism:.4f}")
print(f"RMSE: {rmse:.4f}")
error_report_path = os.path.join(save_dir, "estimator_error_report.csv")
results_df.to_csv(error_report_path, index=False)

In [ ]:
import os, glob
import pandas as pd
import matplotlib.pyplot as plt

pattern = os.path.join(save_dir, f"Nash_Counts_{approach}_nclients_*_maxiter_{max_iters[0]}.csv")
files = sorted(glob.glob(pattern))


counts_all = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
counts_all = counts_all.sort_values("n_clients")


plt.figure(figsize=(10, 6))
plt.plot(counts_all["n_clients"], counts_all["Complete_Occurrences"], marker="o", label="Complete information")
plt.plot(counts_all["n_clients"], counts_all["Lottery_Occurrences"], marker="s", label="Lottery occurrences")
plt.xlabel("Number of clients")
plt.ylabel(f"Occurrences (out of {n_trials})")
plt.title(f"Spmabase FedFor: Complete information vs Lottery rates across clients")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "occurrences_vs_clients_counts.png"), dpi=200)
plt.show()


counts_all["Complete_Rate"]  = counts_all["Complete_Occurrences"]  / n_trials
counts_all["Lottery_Rate"] = counts_all["Lottery_Occurrences"] / n_trials

plt.figure()
plt.plot(counts_all["n_clients"], counts_all["Complete_Rate"], marker="o", label="Complete rate")
plt.plot(counts_all["n_clients"], counts_all["Lottery_Rate"], marker="s", label="Lottery rate")
plt.xlabel("Number of clients")
plt.ylabel("Rate")
plt.ylim(0, 1)
plt.title(f"Spmabase: Complete information vs Lottery rates across clients")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

counts_all
